In [ ]:
# Necessary imports
# can be installed in terminal with -> pip install (name of package)
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
# Default Parameters
# name           value       units           comments
# --------------------------------------------------------------------------------------
A       =       -353;       # W/m^2         OLR intercept
B       =       2.00;       # W/m^2/K       OLR slope / radiative damping coefficient
D       =       3.8;        # W/m^2/K       climate heat transport coefficient; Roughly Budyko's \beta converted to SI
D_ref   =       3.8;        # W/m^2/K       reference value of heat transport coefficient
alpha_0 =       0.3;        # -             albedo when ice-free
alpha_2 =       0.1;        # -             second legendre polynomial coefficient for latitudinal dependence on albedo under ice-free conditions    
dalpha  =       0.22;       # -             increase in albedo when ice-covered
Tf      =       273.15;     # K             temperature threshold for ice
S0      =       1360;       # W/m^2         solar constant, perpendicular incidence
s2      =       -0.48;      # -             second legendre polynomial coefficient for insolation (normalized)  

In [ ]:
def budyko(A, B, D, alpha_0, dalpha, Tf, S0):
    """
    Function to calculate surface albedo feedbacks.
    Done in function form to feed as input to live
    plot with variable inputs.
    """
    yi      = np.linspace(0, 1, 1000);   # sine-latitude of ice edge
    F_yi    = np.zeros(yi.shape);        # decrease in A required to maintain ice line at yi
    ASR_yi  = np.zeros(yi.shape);        # solar absorption with ice-line at yi for reference S0
    Tp_yi   = np.zeros(yi.shape);        # global-mean temperature with ice line at yi
    Tp0_yi  = np.zeros(yi.shape);        # global-mean temperature with ice line at yi and F=0

    insol   = S0/4 * (1 + s2 * 0.5 * (3 * yi**2 - 1));       # annual-mean insolation as a function of latitude
    alpha_warm = alpha_0 + alpha_2 * 0.5 * (3 * yi**2 - 1);  # ice-free albedo
    alpha_cold = alpha_warm + dalpha;                        # ice-covered albedo 

    # For each latitude, calculate ASR(y_i), F(y_i), and T_p(y_i)
    for j in range(len(yi)):
        # Global-mean absorbed solar radiation with ice line at yi
        ASR_yi[j] =\
            yi[j] * np.mean(
                np.multiply(insol[:j+1], 1.0 - alpha_warm[:j+1]) # Below yi
            ) +\
            (1.0-yi[j]) * np.mean(
                np.multiply(insol[j:], 1.0-alpha_cold[j:])       # Above yi
            )

        Tp0_yi[j] = 1.0 / B * (ASR_yi[j] - A)
        num = -1.0 * ( insol[j] * ( 1.0-(alpha_warm[j] + alpha_cold[j])/2 ) - (A + B*Tf) + D*(Tp0_yi[j]-Tf) )
        denom = 1.0 + D/B

        # Radiative forcing required to keep partly ice-covered climate with ice line at yi
        F_yi[j] = num / denom

        # Global-mean temp. for partly ice-covered climate with ice line at yi
        Tp_yi[j] = 1.0 / B * (ASR_yi[j] - A + F_yi[j])

    # Calc. min. possible radiative forcing that'll keep climate ice-free by setting polar T = Tf and using alpha_warm everywhere
    F_min_icefree = -1.0*(insol[-1] * ( 1.0-alpha_warm[-1] ) - (A+B*Tf) + D*(Tp0_yi[-1]-Tf)) / (1.0+D/B)

    # Calc. max possible solar forcing that allows snowball climate by setting equatorial T = Tf and using alpha_cold everywhere
    F_max_snowball = -1.0*(insol[0] * ( 1.0-alpha_cold[0] ) - (A+B*Tf) + D*(Tp0_yi[0]-Tf)) / (1.0+D/B)

    # Stability condition of the Budyko model is that the ice line retreat as the solar forcing increases, so filter calculations for that criterion
    # Pad with leading negative value: model will always be unstable with the ice edge very close to equator
    dyi_dlnS = np.array(np.diff(yi) / np.diff(F_yi))
    dyi_dlnS = np.pad(dyi_dlnS, (1,0), 'constant', constant_values=(-1.0, -1.0))


    ################################
    ########### PLOTTING ###########
    ################################
    plt.rcParams.update({'font.size':9})
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)  # Make 2x2 subplot region
    fig.set_size_inches(6, 7)                           # Change x and y axis size of plot

    # PLOT 1
    pos, neg = dyi_dlnS > 0, dyi_dlnS < 0
    if any(pos):
        ax1.plot([val for b,val in zip(pos, F_yi) if b], [val for b,val in zip(pos, yi) if b], color="b", linewidth=1.5, linestyle="solid")
    if any(neg):
        ax1.plot([val for b,val in zip(neg, F_yi) if b], [val for b,val in zip(neg, yi) if b], color="r", linewidth=1.0, linestyle="dashed")
    
    F_min_plot = min([min(F_yi)-5, F_min_icefree-5]);  # Calculate minimum value of delta for plot
    F_max_plot = max([max(F_yi)+5, F_max_snowball+5]); # and max value of delta for plot
    
    ax1.plot([F_min_plot, F_max_snowball], [0,0], color="#00ebeb", linewidth=1.5)
    ax1.plot([F_min_icefree, F_max_plot], [1,1], color="#ff00f2", linewidth=1.5)
    ax1.plot([0, 0], [0, 1], color='k')
    ax1.set_xlim(left=F_min_plot, right=F_max_plot)
    ax1.set_title('a) Ice line stability diagram')
    ax1.set_xlabel('Global radiative forcing, ' + r'$F=A_0-A$');
    ax1.set_ylabel('Ice-line sine latitude, ' + r'$y_I$');

    # PLOT 2
    ax2.plot(insol, yi, color='k', linewidth=1.5);
    ax2.set_ylabel('sine latitude, y');
    ax2.set_xlabel('insolation ' + r'(W/m$^2$)')
    ax2.set_title('b) Insolation against latitude')

    # PLOT 3
    if any(pos):
        ax3.plot(
            [val for b,val in zip(pos, F_yi) if b], [val for b,val in zip(pos, Tp_yi) if b], 
            color="b", linewidth=1.5, linestyle="solid", label="Stable, partial ice-cover"
        )
    if any(neg):
        ax3.plot(
            [val for b,val in zip(neg, F_yi) if b], [val for b,val in zip(neg, Tp_yi) if b], 
            color="r", linewidth=1.0, linestyle="dashed", label="Unstable, partial ice-cover"
        )
    ax3.plot(
        [F_min_plot, F_max_snowball], 
        [(np.mean(insol * (1.0 - alpha_cold)) - A + F_min_plot)/B, (np.mean(insol * (1.0 - alpha_cold)) - A + F_max_snowball)/B],
        color="#00ebeb", linewidth=1.5, label="Snowball"
    )
    ax3.plot(
        [F_min_icefree, F_max_plot], 
        [(np.mean(insol * (1.0 - alpha_warm)) - A + F_min_icefree)/B, (np.mean(insol * (1.0 - alpha_warm)) - A + F_max_plot)/B],
        color="#ff00f2", linewidth=1.5, label="Ice-free"
    )
    Tmax_plot = (np.mean(insol*(1-alpha_warm))-A+F_max_plot)/B;
    Tmin_plot = (np.mean(insol*(1-alpha_cold))-A+F_min_plot)/B;
    ax3.set_xlabel('Global radiative forcing, ' + r'$F=A_0-A$');
    ax3.set_ylabel('global-mean temperature, K');
    ax3.set_xlim([F_min_plot, F_max_plot]);
    ax3.set_ylim([Tmin_plot, Tmax_plot]);
    ax3.set_title('c )' + r'$\overline{T}$' + ' stability diagram')
    ax3.plot([0, 0],[Tmin_plot, Tmax_plot], color='k'); # line crossing equilibria with no radiative forcing

    # PLOT 4 (Legend and Empty Space)
    ax3.legend(loc='upper right', bbox_to_anchor=(2.3, 1))
    ax4.axis('off')
    plt.subplots_adjust(wspace=0.30, hspace=0.30)
    return


interact(
    budyko, 
    A=(-393.0,-313.0, 1.0), # Set boundaries of sliders here. Center of first two numbers is default setting
    B=(1.0, 3.0, 0.1),
    D=(1.8, 5.9, 0.1),
    alpha_0=(0.0, 0.6, 0.02),
    dalpha=(0.0, 0.44, 0.02),
    Tf=(0.0, 546.3, 1.0),
    S0=(500, 2220, 10),
);